# Code to analyze pbp data for fouls, assists etc

In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

fig_DIR = "../figs/pbp_related/"
img_DIR_P = "../data/images/players/"
data_DIR = "../data/pbpdata/"
export_DIR = "./fdata/"
source = "pbpstats"

In [ ]:
league = "NBA"
season = "2024"
year = season + "-" + str(int(season)+1)[-2:]
with zstd.open(data_DIR + league+"_PBPdata_" + season + ".pkl.zst","rb") as f:
    games_list = dill.load(f)
player_dict = get_players_pbp(league=league)  # Creating Player Dictionary
team_dict = nba_teams.get_teams()  # Creating Team Dictionary

In [ ]:
def rank_data_pbp(IDs, player_dict, team_dict, sort="Player", var="Fouls"):
    ID, items = np.unique(IDs, return_counts=True)
    if sort == "Player":
        ppl = np.array([player_dict.get(x, np.nan) for x in ID])
    elif sort == "Team":
        ppl = [
            team["full_name"] for team, tID in zip(team_dict, ID) if team["id"] == tID
        ]
    df = pd.DataFrame({sort: ppl, "pID": ID,var: items})
    df1 = df.sort_values(by=[var], ascending=False)
    df1["pID"] = df1["pID"].astype(str)
    df1 = df1.reset_index(drop=True).reset_index()
    df1["index"]+=1
    df2 = df1[["index","pID", sort, var]]
    return df2

def plot_table_rank(data,title="",subtitle="",n=10):
    t = (
        GT(data.head(n))
        .tab_header(
            title=md(title),
            subtitle=subtitle
        )
        .tab_source_note(source_note="bsky:@sradjoker.cc | x:@sradjoker | source:pbpstats")
        .cols_label(
            index = "#",
            pID = "",
        )
        .fmt_image(
            columns="pID",
            path = img_DIR_P,
            file_pattern="{}.png"
        )
        .cols_align(align="center")
        .cols_align(align="left", columns="Player")
        .tab_options(
            heading_title_font_size="150%",
            heading_subtitle_font_size="110%",
            # heading_title_font_weight='bold',
            table_background_color="floralwhite",
            column_labels_font_size="105%",
            column_labels_font_weight='bold',
            row_group_font_weight='bold',
            row_group_background_color="#E5E1D8",
            table_font_size=10,
            table_font_names="Consolas", 
            data_row_padding = "3px",
            # table_margin_left = 7,
            # table_margin_right = 0,
        )
    )
    return t

# Frustration Fouls and Euro Fouls

## Estimate Euro Fouls

In [ ]:
possessions = [game.possessions.items for game in games_list]
possession_events = list(chain(*[possession.events for possession in list(chain(*possessions))]))

In [ ]:
t = 5
TO_Miss_pID, TO_Miss_tID, Foul_pID, Foul_tID = [], [], [], []
for possession_event in possession_events:
    if (
        isinstance(possession_event, Foul)
        and (
            isinstance(possession_event.previous_event, Turnover)
            or (
                isinstance(possession_event.previous_event, FieldGoal)
                and not possession_event.previous_event.is_made
            )
        )
        and possession_event.seconds_since_previous_event <= t
    ):
        TO_Miss_pID.append(possession_event.previous_event.player1_id)
        TO_Miss_tID.append(possession_event.previous_event.team_id)
        Foul_pID.append(possession_event.player1_id)
        Foul_tID.append(possession_event.team_id)

In [ ]:
EFoul_tID, EFoul_pID = [], []
for i in range(len(TO_Miss_tID)):
    if TO_Miss_tID[i] == Foul_tID[i]:
        EFoul_tID.append(Foul_tID[i])
    if TO_Miss_pID[i] == Foul_pID[i]:
        EFoul_pID.append(Foul_pID[i])

In [ ]:
data_euro = rank_data_pbp(EFoul_tID, player_dict, team_dict, sort="Team", var="Fouls")
data_euro.to_csv(export_DIR + "NBA_Euro_fouls_"+season+".csv", index=False)
data_euroP = rank_data_pbp(
    EFoul_pID, player_dict, team_dict, sort="Player", var="Fouls"
)
data_euroP.to_csv(export_DIR + "NBA_player_Euro_fouls_"+season+".csv", index=False)

In [ ]:
title = "Euro Fouls " + get_ss(int(season))
subtitle = ""
t = plot_table_rank(data_euroP,title=title,subtitle=subtitle,n=10)
t.save(fig_DIR + f"{league}_euro_foul_"+ season +".png",scale=3,web_driver="firefox")
t

## Estimate Frustration Fouls

In [ ]:
FFoul_pID = []
for i in range(len(TO_Miss_pID)):
    if TO_Miss_pID[i] == Foul_pID[i]:
        FFoul_pID.append(Foul_pID[i])

In [ ]:
data_frust = rank_data_pbp(
    FFoul_pID, player_dict, team_dict, sort="Player", var="Fouls"
)
data_frust.to_csv(export_DIR + "NBA_player_Frustration_fouls_"+ season +".csv", index=False)

In [ ]:
title = "Frustration Fouls " + get_ss(int(season))
subtitle = ""
t = plot_table_rank(data_frust,title=title,subtitle=subtitle,n=10)
t.save(fig_DIR + f"{league}_frust_foul_"+ season +".png",scale=3,web_driver="firefox")
t

# Coast to Coast Analysis

In basketball we say that whenever someone grabs a rebound, dribbles the ball up the court quickly and scores the basket, that the player is going coast to coast.
I came up with a method to estimate these events. I divide these events into two, coast-to-coast buckets i.e. when the player scores by themselves and coast-to-coast assists i.e when the player assists another player to score the ball.
For both types of events I have the same time cutoff of 5 seconds i.e. the player has to score/assist with 5 secs of grabbing the rebound.
Also for coast-to-coast buckets the FG has to bee within 5ft of the rim. NBA considers this distance as a rim attempt.

## Coast to Coast Assists

In [ ]:
t = 5
Rebound_pID, Assist_pID = [], []
i, ii = 0, 0
for possession_event in tqdm(possession_events):
    if (
        (
            isinstance(possession_event, FieldGoal)
            and possession_event.is_made
            and possession_event.is_assisted
        )
        and (isinstance(possession_event.previous_event, Rebound))
        and possession_event.seconds_since_previous_event <= t
    ):
        if (
            possession_event.previous_event.player1_id
            == possession_event.player2_id
        ):
            if not possession_event.previous_event.oreb:
                Rebound_pID.append(possession_event.previous_event.player1_id)
                Assist_pID.append(possession_event.player2_id)

In [ ]:
data = rank_data_pbp(Assist_pID, player_dict, team_dict, sort="Player", var="FGM")
title = "Coast to Coast Assists " + get_ss(int(season))
subtitle = "Player assists within 5s of grabbing rebound"
t = plot_table_rank(data,title=title,subtitle=subtitle,n=10)
t.save(fig_DIR + f"{league}_coast_assists_"+ season +".png",scale=3,web_driver="firefox")
t

## Coast to Coast Buckets

In [ ]:
t = 5
dist = 5
Rebound_pID, FGM_pID = [], []
for possession_event in tqdm(possession_events):
    if (
        (
            isinstance(possession_event, FieldGoal)
            and possession_event.is_made
            and not possession_event.is_assisted
        )
        and (isinstance(possession_event.previous_event, Rebound))
        and possession_event.seconds_since_previous_event <= t
    ):
        if (
            not possession_event.previous_event.oreb
            and possession_event.distance <= dist
        ):
            if not possession_event.previous_event.oreb:
                Rebound_pID.append(possession_event.previous_event.player1_id)
                FGM_pID.append(possession_event.player1_id)

In [ ]:
data_cb = rank_data_pbp(FGM_pID, player_dict, team_dict, sort="Player", var="FGM")

In [ ]:
data = rank_data_pbp(FGM_pID, player_dict, team_dict, sort="Player", var="FGM")
title = "Coast to Coast Buckets " + get_ss(int(season))
subtitle = "Player scores <5ft within 5s of grabbing rebound"
t = plot_table_rank(data,title=title,subtitle=subtitle,n=10)
t.save(fig_DIR + f"{league}_coast_buckets_"+ season +".png",scale=3,web_driver="firefox")
t

# Assists after Offensive rebounds

In [ ]:
ast_time = 14
Rebound_pID, Assist_pID = [], []
for possession_event in tqdm(possession_events):
    if (
        isinstance(possession_event, FieldGoal)
        and possession_event.is_made
        and possession_event.is_assisted
    ) and (isinstance(possession_event.previous_event, Rebound)):
        if (
            possession_event.previous_event.player1_id
            == possession_event.player2_id
        ):
            if possession_event.previous_event.oreb:
                Rebound_pID.append(possession_event.previous_event.player1_id)
                Assist_pID.append(possession_event.player2_id)

In [ ]:
data_oa = rank_data_pbp(Assist_pID, player_dict, team_dict, sort="Player", var="AST")

In [ ]:
title = "Off Reb Assists " + year
footer = "Player assists after grabbing off rebound"
fig = plot_table_rank(data_oa, "AST", title=title, footer=footer, source="pbpstats")
fig.write_image(fig_DIR + f"{league}_oreb_assists_"+ season +".png", scale=3)

# Loose Ball Foul

In [ ]:
possa = []
Foul_pID = []
for possession_event in possession_events:
    if isinstance(possession_event, Foul):
        if possession_event.is_loose_ball_foul:
            possa.append(possession_event)
            Foul_pID.append(possession_event.player1_id)

In [ ]:
data_Foul = rank_data_pbp(
    Foul_pID, player_dict, team_dict, sort="Player", var="Fouls"
)
title = "Loose Ball Fouls " + "2023-24"
fig = plot_table_rank(data_Foul, "Fouls", sort="Player", title=title, source="pbpstats")
fig.write_image(fig_DIR + f"{league}_loose_foul_"+ season +".png", scale=3)

# Offensive Foul

In [ ]:
possa = []
Foul_pID = []
for possession_event in possession_events:
    if isinstance(possession_event, Foul):
        if possession_event.is_offensive_foul:
            possa.append(possession_event)
            Foul_pID.append(possession_event.player1_id)

In [ ]:
data_Foul = rank_data_pbp(
    Foul_pID, player_dict, team_dict, sort="Player", var="Fouls"
)
title = "Offensive Fouls " + "2023-24"
fig = plot_table_rank(data_Foul, "Fouls", sort="Player", title=title, source="pbpstats")
fig.write_image(fig_DIR + f"{league}_off_foul_"+ season +".png", scale=3)

# Chase Down Blocks

In [ ]:
possessions = [game.possessions.items for game in games_list]
possession_events = list(chain(*[possession.events for possession in list(chain(*possessions))]))
t = 5
dist = 5
Block_pID, FGA_pID = [], []
for possession_event in possession_events:
    if (
        (
            isinstance(possession_event, FieldGoal)
            and not possession_event.is_made
        )
        and possession_event.is_blocked
        and possession_event.seconds_since_previous_event <= t
        and possession_event.distance <= dist
    ):
        if (
            isinstance(possession_event.previous_event, Rebound)
            and possession_event.previous_event.oreb
        ):
            pass
        else:
            Block_pID.append(possession_event.player3_id)
            FGA_pID.append(possession_event.player1_id)

In [ ]:
data = rank_data_pbp(Block_pID, player_dict, team_dict, sort="Player", var="Block")
title = "Chase Down Blocks Leaders: 2024-25"
subtitle = "Block <5 feet of rim and within 5s of poss change"
t = plot_table_rank(data,title=title,subtitle=subtitle,n=10)
t.save(fig_DIR + f"{league}_chasedown_"+ season +".png",scale=3,web_driver="firefox")
t

# Block and Score

In [ ]:
possessions = [game.possessions.items for game in games_list]
possession_events = list(chain(*[possession.events for possession in list(chain(*possessions))]))
poss = []
Block_pID, FGA_pID = [], []
for possession_event in possession_events:
    if (
        (
            isinstance(possession_event, FieldGoal)
            and not possession_event.is_made
        )
        and possession_event.is_blocked
    ):  
        if (
            isinstance(possession_event.next_event.next_event, FieldGoal)
            and possession_event.next_event.next_event.is_made
        ): 
            # print(possession_event.player3_id)
            Block_pID.append(possession_event.player3_id)
            FGA_pID.append(possession_event.next_event.next_event.player1_id)
            poss.append(possession_event)
        else:
            pass

In [ ]:
BScore_pID = []
BScore_poss = []
for i in range(len(Block_pID)):
    if Block_pID[i] == FGA_pID[i]:
        # print(Block_pID)
        BScore_pID.append(Block_pID[i])
        BScore_poss.append(poss[i])

In [ ]:
data = rank_data_pbp(BScore_pID, player_dict, team_dict, sort="Player", var="BLK")
title = "Block + Score Immediately" 
subtitle = get_ss(int(season)) + " Season, as of "+ datetime.today().strftime("%Y-%m-%d")
t = plot_table_rank(data,title=title,subtitle=subtitle,n=10)
t.save(fig_DIR + f"{league}_block_score_"+ season +".png",scale=3,web_driver="firefox")
t

# Steal and Score

In [ ]:
possessions = [game.possessions.items for game in games_list]
possession_events = list(chain(*[possession.events for possession in list(chain(*possessions))]))
poss = []
Steal_pID, FGA_pID = [], []
for possession_event in possession_events:
    if (
        (
            isinstance(possession_event, Turnover)
        )
        and possession_event.is_steal
    ):  
        if (
            isinstance(possession_event.next_event, FieldGoal)
            and possession_event.next_event.is_made
            and not possession_event.next_event.is_assisted
        ): 
            # print(possession_event.player3_id)
            Steal_pID.append(possession_event.player3_id)
            FGA_pID.append(possession_event.next_event.player1_id)
            poss.append(possession_event)
        else:
            pass

In [ ]:
SScore_pID = []
SScore_poss = []
for i in range(len(Steal_pID)):
    if Steal_pID[i] == FGA_pID[i]:
        # print(Steal_pID)
        SScore_pID.append(Steal_pID[i])
        SScore_poss.append(poss[i])

In [ ]:
data = rank_data_pbp(SScore_pID, player_dict, team_dict, sort="Player", var="STL")
title = "Steal + Score Immediately" 
subtitle = get_ss(int(season)) + " Season, as of "+ datetime.today().strftime("%Y-%m-%d")
t = plot_table_rank(data,title=title,subtitle=subtitle,n=10)
# t.save(fig_DIR + f"{league}_steal_score_"+ season +".png",scale=3,web_driver="firefox")
t.save(fig_DIR + f"{league}_steal_score_ua_"+ season +".png",scale=3,web_driver="firefox")
t